In [4]:
import ipywidgets as widgets
from ipywidgets import interact
from src.post_processing import PathWrangler
from IPython.display import display, SVG
import numpy as np
import matplotlib.pyplot as plt

In [80]:
path_filepath = '../artifacts/processed_expansions/found_paths.json'
predicted_reactions_filepath = "../artifacts/processed_expansions/predicted_reactions.json"
known_reactions_filepath = "../artifacts/processed_expansions/known_reactions.json"

pw = PathWrangler(
    path_filepath=path_filepath,
    pr_filepath=predicted_reactions_filepath,
    kr_filepath=known_reactions_filepath
)

In [104]:
evidence_options = {elt.value: elt.name for elt in pw.enzyme_existence}
default_loe = pw.enzyme_existence.PROTEIN.value
loe_selection = [evidence_options[default_loe]]

enzyme_evidence = widgets.SelectMultiple(
    options=evidence_options.keys(),
    value=[default_loe],
    rows=len(evidence_options),
    description='Enzyme level of evidence:',
    disabled=False
)

def evidence_listener(change):
    global loe_selection
    loe_selection = [evidence_options[k] for k in enzyme_evidence.value]

enzyme_evidence.observe(evidence_listener)

enzyme_evidence

SelectMultiple(description='Enzyme level of evidence:', index=(0,), options=('Evidence at protein level', 'Evi…

In [105]:
sort_by_selection = 'mean_rcmcs'
sort_options = ['mean_rcmcs', 'min_rcmcs', 'mdf']
sort_default = 'mean_rcmcs'

sort_by_radio_buttons = widgets.RadioButtons(
    options=sort_options,
    value=sort_default,
    description='Sort paths by:',
    disabled=False
)

def sort_by_listener(change):
    global sort_by_selection
    sort_by_selection = sort_by_radio_buttons.value

sort_by_radio_buttons


RadioButtons(description='Sort paths by:', options=('mean_rcmcs', 'min_rcmcs', 'mdf'), value='mean_rcmcs')

In [91]:
paths = []
def click_handler(b):
    global paths
    filter_by_enzymes = {'existence': loe_selection} # Only paths w/ enzymes w/ protein-level evidence
    paths = pw.get_paths(
        sort_by=sort_by_selection,
        filter_by_enzymes=filter_by_enzymes
    )
    print(f"{len(paths)} paths meet your criteria")

button = widgets.Button(
    description='Get paths',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='solid flask' # (FontAwesome names without the `fa-` prefix)
)

button.on_click(click_handler)

button

Button(button_style='success', description='Get paths', icon='solid flask', style=ButtonStyle(), tooltip='Clic…

11 paths meet your criteria
11 paths meet your criteria
11 paths meet your criteria
15 paths meet your criteria
7 paths meet your criteria
7 paths meet your criteria
15 paths meet your criteria


In [108]:
path_slider = widgets.IntSlider(
    min=0,
    max=len(paths) - 1,
    value=0,
    description="Path # "
)

@interact
def display_path(i=path_slider):
    rxn_imgs = (reaction.image for reaction in paths[i].reactions)
    for elt in rxn_imgs: display(SVG(elt))

interactive(children=(IntSlider(value=0, description='Path # ', max=14), Output()), _dom_classes=('widget-inte…